# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 9:47AM,239228,21,399290089,"NBTY Global, Inc.",Released
1,Jun 9 2022 9:44AM,239227,10,MET211076,"Methapharm, Inc.",Released
2,Jun 9 2022 9:41AM,239226,10,MET211085,"Methapharm, Inc.",Released
3,Jun 9 2022 9:37AM,239225,19,ELSE61022,Else Nutrition,Released
4,Jun 9 2022 9:34AM,239224,10,NST-1050,"Amcyte Pharma, Inc.",Released
5,Jun 9 2022 9:29AM,239223,20,EXACT-8121986,"Exact-Rx, Inc.",Released
6,Jun 9 2022 9:29AM,239222,10,BPS1248,Beach Patient Assistance,Released
7,Jun 9 2022 9:29AM,239222,10,BPS1249,Beach Patient Assistance,Released
8,Jun 9 2022 9:29AM,239222,10,BPS1250,Beach Patient Assistance,Released
9,Jun 9 2022 9:29AM,239222,10,BPS1251,Beach Patient Assistance,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
55,239224,Released,1
56,239225,Released,1
57,239226,Released,1
58,239227,Released,1
59,239228,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239224,NaN,NaN,1.0
239225,NaN,NaN,1.0
239226,NaN,NaN,1.0
239227,NaN,NaN,1.0
239228,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239107,0.0,0.0,1.0
239110,0.0,1.0,0.0
239122,0.0,0.0,1.0
239124,4.0,11.0,4.0
239134,0.0,12.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239107,0,0,1
239110,0,1,0
239122,0,0,1
239124,4,11,4
239134,0,12,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239107,0,0,1
1,239110,0,1,0
2,239122,0,0,1
3,239124,4,11,4
4,239134,0,12,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239107,,,1
1,239110,,1,
2,239122,,,1
3,239124,4,11,4
4,239134,,12,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc."
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc."
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc."
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc."
5,Jun 9 2022 9:29AM,239223,20,"Exact-Rx, Inc."
6,Jun 9 2022 9:29AM,239222,10,Beach Patient Assistance
13,Jun 9 2022 9:25AM,239221,12,Hush Hush
27,Jun 9 2022 9:16AM,239220,20,"Exact-Rx, Inc."
29,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc.",,,1
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc.",,,1
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc.",,,1
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition,,,1
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc.",,,1
5,Jun 9 2022 9:29AM,239223,20,"Exact-Rx, Inc.",,,1
6,Jun 9 2022 9:29AM,239222,10,Beach Patient Assistance,,,7
7,Jun 9 2022 9:25AM,239221,12,Hush Hush,,,14
8,Jun 9 2022 9:16AM,239220,20,"Exact-Rx, Inc.",,,2
9,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc.",1,,
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc.",1,,
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc.",1,,
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition,1,,
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc.",1,,
5,Jun 9 2022 9:29AM,239223,20,"Exact-Rx, Inc.",1,,
6,Jun 9 2022 9:29AM,239222,10,Beach Patient Assistance,7,,
7,Jun 9 2022 9:25AM,239221,12,Hush Hush,14,,
8,Jun 9 2022 9:16AM,239220,20,"Exact-Rx, Inc.",2,,
9,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc.",1,,
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc.",1,,
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc.",1,,
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition,1,,
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition,1.0,NaN,NaN
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:47AM,239228,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 9 2022 9:44AM,239227,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Jun 9 2022 9:41AM,239226,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 9 2022 9:37AM,239225,19,Else Nutrition,1.0,0.0,0.0
4,Jun 9 2022 9:34AM,239224,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5023072,191.0,6.0,26.0
12,239221,14.0,0.0,0.0
16,956566,2.0,2.0,0.0
18,956703,4.0,0.0,0.0
19,2152688,35.0,22.0,35.0
20,1195916,22.0,23.0,4.0
21,717541,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5023072,191.0,6.0,26.0
1,12,239221,14.0,0.0,0.0
2,16,956566,2.0,2.0,0.0
3,18,956703,4.0,0.0,0.0
4,19,2152688,35.0,22.0,35.0
5,20,1195916,22.0,23.0,4.0
6,21,717541,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,191.0,6.0,26.0
1,12,14.0,0.0,0.0
2,16,2.0,2.0,0.0
3,18,4.0,0.0,0.0
4,19,35.0,22.0,35.0
5,20,22.0,23.0,4.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,191.0
1,12,Released,14.0
2,16,Released,2.0
3,18,Released,4.0
4,19,Released,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,26.0,0.0,0.0,0.0,35.0,4.0,0.0
Executing,6.0,0.0,2.0,0.0,22.0,23.0,0.0
Released,191.0,14.0,2.0,4.0,35.0,22.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,35.0,4.0,0.0
1,Executing,6.0,0.0,2.0,0.0,22.0,23.0,0.0
2,Released,191.0,14.0,2.0,4.0,35.0,22.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,35.0,4.0,0.0
1,Executing,6.0,0.0,2.0,0.0,22.0,23.0,0.0
2,Released,191.0,14.0,2.0,4.0,35.0,22.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()